In [1]:
import gc

import torch
import pandas as pd

from unsloth import FastLanguageModel
from datasets import Dataset
from sklearn.model_selection import train_test_split
from trl import SFTTrainer
from transformers import TrainingArguments, EarlyStoppingCallback
from unsloth import is_bfloat16_supported

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/opt/miniconda3/envs/project_news_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth Zoo will now patch everything to make training faster!


In [2]:
df_path = '{your_dataset_path}'

main_df = pd.read_csv(df_path)

main_df.drop(columns = ['Unnamed: 0', 'Тэги', 'Дата публикации'], inplace = True)
#Удаляем колонки, которые не будут задействованы в обучении

model_instructions = """
    Ты - профессиональный журналист с многолетним опытом.
    Твоя задача - сгенерировать заголовок новостной статьи, который в максимальной степени отражал бы содержание новости.
    Не поясняй свой ответ. Твой финальный ответ должен включать в себя только заголовок и ничего более.
    """
main_df['Инструкции модели'] = model_instructions

train_dataset, test_dataset = train_test_split(main_df, test_size = 0.1, random_state = 1337)

train_dataset = train_dataset.reset_index().drop(columns = ['index'])
test_dataset = test_dataset.reset_index().drop(columns = ['index'])


print(f'Количество рядов в тренировочном датасете: {len(train_dataset)}\n\
      Количество рядов в тестовом датасете: {len(test_dataset)}')

Количество рядов в тренировочном датасете: 4373
      Количество рядов в тестовом датасете: 486


In [3]:
max_seq_length = 1048
dtype = None
load_in_4bit = True

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = 'unsloth/gemma-2-2b',
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit
)

In [5]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, #Стандартное значение 16, однако поскольку модель маленькая
    lora_alpha = 16, #Scaling factor для обновления LoRa - обычно равен R для балансированного обновления весов.
    lora_dropout = 0, 
    bias = 'none',
    use_gradient_checkpointing = 'unsloth',
    random_state = 1337,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2024.12.4 patched 26 layers with 26 QKV layers, 26 O layers and 26 MLP layers.


In [3]:
alpaca_prompt = """Ниже представлены следующие аспекты:
1. Пользовательский запрос.
2. Исходные данные - контекст
3. Ответ - твой ответ.
Сгенерируй ответ, который в полной мере выполняет пользовательский запрос.  
### Пользовательский запрос:
{}

### Исходные данные:
{}

### Ответ:
{}"""

In [ ]:
EOS_TOKEN = tokenizer.eos_token #EOS или токен окончания последовательности, добавляется, чтобы избежать бесконечных генераций.

In [8]:
def format_prompt(examples):
    instructions = examples['Инструкции модели']
    input = examples['Текст новости']
    output = examples['Заголовок']

    texts = []

    for instructions, input, output in zip(instructions, input, output):
        text = alpaca_prompt.format(instructions, input, output) + EOS_TOKEN
        texts.append(text)
    return {'text': texts}

In [9]:
train_dataset = Dataset.from_pandas(train_dataset)
test_dataset = Dataset.from_pandas(test_dataset)

train_dataset = train_dataset.map(format_prompt, batched = True)
test_dataset = test_dataset.map(format_prompt, batched = True)

Map:   0%|          | 0/4373 [00:00<?, ? examples/s]

Map: 100%|██████████| 486/486 [00:00<00:00, 42752.34 examples/s]


Model FineTuning

In [10]:
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    eval_dataset = test_dataset,
    dataset_text_field = 'text',
    callbacks = [EarlyStoppingCallback(early_stopping_patience=5)], #Добавляем ES == 5 эпохам.
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,


    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 4000,
        eval_strategy = 'steps',
        metric_for_best_model = 'eval_loss',
        load_best_model_at_end = True,
        greater_is_better = False,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 100,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 1337,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    )
)

Map (num_proc=2): 100%|██████████| 486/486 [00:01<00:00, 311.13 examples/s]


In [11]:
save_path = '{insert_your_gemma2_2b_save_path}'

In [12]:
trainer_stats = trainer.train()

model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 4,373 | Num Epochs = 8
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 4,000
 "-____-"     Number of trainable parameters = 20,766,720


Step,Training Loss,Validation Loss
100,1.450200,2.142085
200,1.372700,2.085290
300,1.368100,2.041223
400,1.320200,2.026646
500,1.310900,2.009410
600,1.265800,2.035812
700,1.203600,2.015994
800,1.170000,2.031903
900,1.170300,2.026358
1000,1.202000,2.017031


('gemma2_2b_finetuned/tokenizer_config.json',
 'gemma2_2b_finetuned/special_tokens_map.json',
 'gemma2_2b_finetuned/tokenizer.model',
 'gemma2_2b_finetuned/added_tokens.json',
 'gemma2_2b_finetuned/tokenizer.json')

Инференс модели после дообучения

In [13]:
FastLanguageModel.for_inference(model)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Gemma2ForCausalLM(
      (model): Gemma2Model(
        (embed_tokens): Embedding(256000, 2304, padding_idx=0)
        (layers): ModuleList(
          (0-25): 26 x Gemma2DecoderLayer(
            (self_attn): Gemma2Attention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=2304, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2304, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora

In [19]:
inputs = tokenizer(
    [
        alpaca_prompt.format(train_dataset['Инструкции модели'][0],
                             test_dataset['Текст новости'][3],
                             "") #Response left blank for generation
    ], return_tensors = 'pt').to('cuda')

outputs = model.generate(**inputs, max_new_tokens = 100, use_cache = True, pad_token_id = tokenizer.eos_token_id)
result = tokenizer.batch_decode(outputs)

In [26]:
print(f"Заголовок, сгенерированный моделью: {result[0].split('Ответ:', 1)[1]}\nОбразцовый заголовок:\n{test_dataset['Заголовок'][3]}")

Заголовок, сгенерированный моделью: 
Томскстат назвал продукты, которые подорожали в июне, а также услуги<eos>
Образцовый заголовок:
Как изменились цены на продукты и услуги в Томске за месяц: данные статистики


Модель сгенерировала вполне приемлимый заголовок

Опишем функцию для автоматизации дообучения

In [3]:
def train_model(model: str, train_set: pd.DataFrame, eval_set: pd.DataFrame) -> None:
    '''Функция выполняет дообучение указанной модели из библиотеки Unsloth.
    Предполагаемое использование - дообучение больших языковых моделей для генерации новостных заголовков
    
    Args:
        model (str): Название модели, совместимое с библиотекой Unsloth.
        train_set (pd.DataFrame): Датафрейм, содержащий в себе следующие колонки:\
            1. Инструкции модели;
            2. Текст новости;
            3. Заголовок
        test_set (pd.DataFrame): Датафрейм структурно идентичный train_set
    Returns:
        None - Функция не возращает конкретного значения:\
        По завершению дообучения модели, модель и токкенизатор будут записаны в директорию,\
        название которой соответсвует названию выбранной для дообучения модели.
    '''
    
    model_name = model

    
    print(f'Выполняется инициализация модели {model_name}...')

    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = model,
        max_seq_length = 1048,
        dtype = None,
        load_in_4bit = True
    )
    
    def clear_cache(model, tokenizer):
        '''Функция удаляет модель из GPU'''
        with torch.no_grad():
            model.cpu()
        del model
        del tokenizer
    
        gc.collect()
    
        torch.cuda.empty_cache()
    

    EOS_TOKEN = tokenizer.eos_token


    alpaca_prompt = """Ниже представлены следующие аспекты:
    1. Пользовательский запрос.
    2. Исходные данные - контекст
    3. Ответ - твой ответ.
    Сгенерируй ответ, который в полной мере выполняет пользовательский запрос.  
    ### Пользовательский запрос:
    {}

    ### Исходные данные:
    {}

    ### Ответ:
    {}"""

    def format_prompt(examples):
        '''Функция форматирует alpaca_prompt_template в соответствии с пользовательский задачей.
        
        Args:
            examples (pandas.dataframe): Датафрейм, содержащий колонки:
                                        1. Инструкции модели.
                                        2. Текст новости.
                                        3. Заголовок
        
        Returns:
            dict - Словарь: ключ - строка 'text', значение - отформатированный prompt_template
        '''
        instructions = examples['Инструкции модели']
        input = examples['Текст новости']
        output = examples['Заголовок']

        texts = []

        for instructions, input, output in zip(instructions, input, output):
            text = alpaca_prompt.format(instructions, input, output) + EOS_TOKEN
            texts.append(text)
        
        return {'text': texts}
    
    
    train_dataset = Dataset.from_pandas(train_set)
    test_dataset = Dataset.from_pandas(eval_set)

    train_dataset = train_dataset.map(format_prompt, batched = True)
    test_dataset = test_dataset.map(format_prompt, batched = True)

    model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    lora_alpha = 16,
    lora_dropout = 0, 
    bias = 'none',
    use_gradient_checkpointing = 'unsloth',
    random_state = 1337,
    use_rslora = False,
    loftq_config = None,
)
    trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    eval_dataset = test_dataset,
    dataset_text_field = 'text',
    callbacks = [EarlyStoppingCallback(early_stopping_patience=5)], #Добавляем ES == 5 эпохам.
    max_seq_length = 1048,
    dataset_num_proc = 2,
    packing = False,
    
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 5000,
        eval_strategy = 'steps',
        metric_for_best_model = 'eval_loss',
        load_best_model_at_end = True,
        greater_is_better = False,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 100,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 1337,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    )
)
    save_path = model_name

    print(f'Начинается процесс дообучение модели {model_name}...')

    trainer.train()

    model.save_pretrained(save_path)
    tokenizer.save_pretrained(save_path)

    clear_cache(model = model, tokenizer = tokenizer)

Опишем цикл for для автоматизации дообучения.

In [ ]:
selected_models = ['unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit',
                   'unsloth/gemma-2-9b-bnb-4bit']

for model in selected_models:
    
    train_model(train_set = train_dataset,
                eval_set = test_dataset,
                model = model)
    
    print(f"Дообучение модели {model} завершено.")
